In [1]:
import pandas as pd

df=pd.read_csv("output.csv")

df['Keywords'] = df['Keywords'].str.split(',')
df['GroundTruth Keywords']=df['GroundTruth Keywords'].str.split(',')

df_exploded_k = df.explode('Keywords')
df_exploded_gk = df.explode('GroundTruth Keywords')

keywords_list = df_exploded_k['Keywords'].str.strip().tolist()
groundtruth_list = df_exploded_gk['GroundTruth Keywords'].tolist()

In [3]:
def calculate_jaccard_similarity(list1, list2):
    # Lowercase all keywords
    list1_lower = [keyword.lower() for keyword in list1]
    list2_lower = [keyword.lower() for keyword in list2]
    
    # Convert lists to sets
    set1 = set(list1_lower)
    set2 = set(list2_lower)
    
    # Calculate intersection and union
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    # Calculate Jaccard similarity
    jaccard_similarity = len(intersection) / len(union)
    
    return jaccard_similarity

#
similarity_score = calculate_jaccard_similarity(groundtruth_list, keywords_list)
print("Jaccard Similarity:", round(similarity_score*10))

Jaccard Similarity: 7


In [ ]:
import gensim.downloader as api
word2vec_model = api.load('word2vec-google-news-300')
def get_average_embedding(keyword_list, word2vec_model):
    embeddings = []
    for keyword in keyword_list:
        if keyword in word2vec_model:
            embeddings.append(word2vec_model[keyword])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

groundtruth_embedding = get_average_embedding(groundtruth_list, word2vec_model)
model_embedding = get_average_embedding(keywords_list, word2vec_model)


similarity_score = cosine_similarity(groundtruth_embedding, model_embedding)
print(similarity_score)
similarity_percentage = similarity_score * 100
print(f"Word2Vec cosine similarity: {similarity_percentage:.2f}%")